# 画像ファイルに対応するキロ程紐づけ用のJSONファイルを作成するノートブック
```
以下のコマンドを順番に実行していく
想定しているディレクトリ構造は以下のとおり
.
├ imgs
│  └ dir_area    :線区単位の画像を保存
│     ├ HD11     :カメラ毎の画像を保存
│     ├ HD12     :以下、同じ
│     ├ HD21         
│     ├ HD21 
│     ├ HD31 
│     └ HD32 
├ TDM
│  └ TDM_GazoFileIndex.xlsx  :車モニから出力したマスターデータ
└ imgKiro.ipynb  :本ファイル
```

# 一括で線区フォルダごとのJSONファイルを作成する

## 1. ライブラリ読込み＆関数の定義

In [3]:
import importlib
from pprint import pprint
from tqdm.notebook import tqdm
import boto3

import gc
import re
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw, ImageFont
from src.config import appProperties
import src.helpers as helpers
import src.visualize as vis
importlib.reload(helpers)
importlib.reload(vis)

config = appProperties('config.yml')

# 駅コードのjsonファイル                                        # 2024.8.13
path = path = f"{config.tdm_dir}/eki_code.json"
with open(path, encoding="utf-8") as f:
    eki_code =  json.load(f)

def get_df(tdm_fpath):
    """ 車モニのマスタデータを読み込む
    Args:
        tdm_fpath(str): 車モニ マスターデータのパス
    Return:
        df_tdm(DataFrame): 読み込まれたデータフレーム
    """
    # カラム名をS3から読み込む
    bucket_name = 'trolley-monitor'
    columns_csv_key = 'img-location-table/TDM_GazoFileIndex_ColumnsName.csv'
    columns = get_columns_from_csv(bucket_name, columns_csv_key)
    
    # TDM(車モニCSV)をS3からダウンロードする
    download_file_from_s3(bucket_name, csv_file, download_path)
    print("車モニのマスターデータを読み込みます ※少し時間がかかります")
    print(f'complete: {download_path=}')
    
    # CSVからデータフレームを読み込む
    print("車モニのマスターデータを読み込みます ※少し時間がかかります")
    # df_tdm = pd.read_excel(tdm_fpath)
    df_tdm = pd.read_csv(tdm_fpath, names=columns, delimiter='|')
    
    # データフレームを整形する
    df_tdm['TimeCode'] = pd.to_datetime(df_tdm['TimeCode'])
    df_tdm = df_tdm.sort_values(by=['SokuteiDate', 'KiroTei'], ignore_index=True)
    print("車モニのマスターデータを読み込みました")
    print(f"データフレームのサイズ:{df_tdm.shape}")
    # 欲しい列だけ抽出する
    df_tdm = df_tdm.filter([
        'SokuteiYear', 'NennaiSeqNo', 'EkiCd', 'SenbetsuCd', 'SokuteiDate', 'DenchuNo', 'KiroTei',
        'GazoFileNameHD11', 'GazoFileNameHD12',
        'GazoFileNameHD21', 'GazoFileNameHD22',
        'GazoFileNameHD31', 'GazoFileNameHD32'
    ]).copy()
    print("必要な情報だけフィルタリングしました")
    print(f"データフレームのサイズ:{df_tdm.shape}")
    return df_tdm

def get_df2(bucket_name, csv_file, columns):                                    # 2024.08.12
    """ 車モニのマスタデータを読み込む
    Args:
        tdm_fpath(str): 車モニ マスターデータのパス
    Return:
        df_tdm(DataFrame): 読み込まれたデータフレーム
    """
    
    # TDM(車モニCSV)をS3からダウンロードする
    download_path = f"TDM/temp/{csv_file.split('/')[-1]}"
    download_file_from_s3(bucket_name, csv_file, download_path)                  # 無駄に時間かかるのでコメントアウト（後で戻す）
    print("車モニのマスターデータを読み込みます ※少し時間がかかります")               # 2024.08.12 毎回printされてしまう
    print(f'complete: {download_path=}')

    # CSVからデータフレームを読み込む
    df_tdm = pd.read_csv(download_path, names=columns, delimiter='|')

    # データフレームを整形する
    df_tdm['TimeCode'] = pd.to_datetime(df_tdm['TimeCode'])
    df_tdm = df_tdm.sort_values(by=['SokuteiDate', 'KiroTei'], ignore_index=True)
    print("車モニのマスターデータを読み込みました")
    print(f"データフレームのサイズ:{df_tdm.shape}")

    # 欲しい列だけ抽出する
    df_tdm = df_tdm.filter([
        'EkiCd', 'SenbetsuCd', 'SokuteiDate', 'DenchuNo', 'KiroTei',               # Comment Out    'SokuteiYear' 'NennaiSeqNo'は追加されている
        'GazoFileNameHD11', 'GazoFileNameHD12',
        'GazoFileNameHD21', 'GazoFileNameHD22',
        'GazoFileNameHD31', 'GazoFileNameHD32'
    ]).copy()

    print("必要な情報だけフィルタリングしました")
    print(f"データフレームのサイズ:{df_tdm.shape}")
    return df_tdm

def get_df3(bucket_name, csv_file, columns):                                    # 2024.08.13
    """ 車モニのマスタデータを読み込む
    Args:
        tdm_fpath(str): 車モニ マスターデータのパス
    Return:
        df_tdm(DataFrame): 読み込まれたデータフレーム
    """
    
    # TDM(車モニCSV)をS3からダウンロードする
    download_path = f"TDM/temp/{csv_file.split('/')[-1]}"
    download_file_from_s3(bucket_name, csv_file, download_path)                    # 無駄に時間かかるのでコメントアウト（後で戻す）
    print("車モニのマスターデータを読み込みます ※少し時間がかかります")               # 2024.08.12 毎回printされてしまう
    print(f'complete: {download_path=}')

    # CSVからデータフレームを読み込む
    df_tdm = pd.read_csv(download_path, names=columns, delimiter='|')

    # データフレームを整形する
    df_tdm['TimeCode'] = pd.to_datetime(df_tdm['TimeCode'])
    df_tdm = df_tdm.sort_values(by=['SokuteiDate', 'KiroTei'], ignore_index=True)
    print("車モニのマスターデータを読み込みました")
    print(f"データフレームのサイズ:{df_tdm.shape}")

    # 欲しい列だけ抽出する
    df_tdm = df_tdm.filter([
        'EkiCd', 'SenbetsuCd', 'SokuteiDate', 'DenchuNo', 'KiroTei',               # Comment Out    'SokuteiYear' 'NennaiSeqNo'は追加されている
        'GazoFileNameHD11', 'GazoFileNameHD12',
        'GazoFileNameHD21', 'GazoFileNameHD22',
        'GazoFileNameHD31', 'GazoFileNameHD32'
    ]).copy()

    print("必要な情報だけフィルタリングしました")
    print(f"データフレームのサイズ:{df_tdm.shape}")
    return df_tdm


def list_csv_files(bucket_name, prefix):
    """ S3にあるCSVの一覧をゲット
    """
    s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')

    csv_files = []
    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
        if 'Contents' in page:
            for obj in page['Contents']:
                if obj['Key'].endswith('.csv'):
                    csv_files.append(obj['Key'])

    return csv_files


def download_file_from_s3(bucket_name, key, download_path):
    """ S3からダウンロードする
    """
    # print(f"{bucket_name=} {key=} {download_path=}")
    s3 = boto3.client('s3')
    s3.download_file(bucket_name, key, download_path)


def get_columns_from_csv(bucket_name, columns_csv_key):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=columns_csv_key)
    columns_df = pd.read_csv(obj['Body'])
    return columns_df.columns.tolist()


def get_KiroTei_dict(config, df):
    """ 走行日・線区ごとのキロ程補正情報を辞書に記録する
    """
    # カメラ番号ごとに、EkiCdのはじめの行をTrueとして記録する
    for camera_num in config.camera_types:
        df[f"EkiCdDiff{camera_num}"] = df.query(f"GazoFileName{camera_num}.notnull()")['EkiCd'].diff().map(lambda x: x != 0)
    # 走行日・線区ごとに、画像1枚あたりの幅(m)を算出して辞書に記録する
    result_dict = {}
    for camera_num in config.camera_types:
        # grouped_df = df.query(f"GazoFileName{camera_num}.notnull()").groupby(['SokuteiDate', 'EkiCd'])['KiroTei_NEWSS'].count()             # 2024.6.24 Yamazwa coment out
        grouped_df = df.query(f"GazoFileName{camera_num}.notnull()").groupby(['SokuteiDate', 'EkiCd'])['KiroTei'].count()               # 2024.6.24 Yamazwa add
        for (date, ekicd), count in grouped_df.items():
            if date not in result_dict:
                result_dict[date] = {}
            if ekicd not in result_dict[date]:
                result_dict[date][ekicd] = {}
            # print(f"camera_num: {camera_num}, date: {date}, ekicd: {ekicd}")
            # KiroTei_head = df.query(f"EkiCdDiff{camera_num}.notnull() & SokuteiDate == {date} & EkiCd == {ekicd}")['KiroTei_NEWSS'].iloc[0]    # 2024.6.24 Yamazwa coment out
            # KiroTei_tail = df.query(f"EkiCdDiff{camera_num}.notnull() & SokuteiDate == {date} & EkiCd == {ekicd}")['KiroTei_NEWSS'].iloc[-1]   # 2024.6.24 Yamazwa coment out
            KiroTei_head = df.query(f"EkiCdDiff{camera_num}.notnull() & SokuteiDate == {date} & EkiCd == {ekicd}")['KiroTei'].iloc[0]            # 2024.6.24 Yamazwa add
            KiroTei_tail = df.query(f"EkiCdDiff{camera_num}.notnull() & SokuteiDate == {date} & EkiCd == {ekicd}")['KiroTei'].iloc[-1]           # 2024.6.24 Yamazwa add
            result_dict[date][ekicd][camera_num] = {
                'KiroTei_head': KiroTei_head,
                'KiroTei_tail': KiroTei_tail,
                'KiroTei_dist': round(KiroTei_tail - KiroTei_head, 4),
                'KiroTei_delta': round((KiroTei_tail - KiroTei_head) / (count - 1), 6),
                'count': count
            }
    return df, result_dict


def get_Kiro_offset(df_tdm, ref_point, date):
    """ 基準位置を指定し、検測キロ程をNEWSSキロ程に変換するオフセット値を取得する
    Args:
    
    Return:
    
    """
    pole_num_matched = df_tdm.query(
        f"EkiCd == {ref_point['EkiCd_NEWSS']} & SokuteiDate == {date} & DenchuNo == {ref_point['pole_num_NEWSS']}"
    ).index.tolist()
    pole_kiro_twins = df_tdm.loc[min(pole_num_matched)]['KiroTei']
    pole_kiro_offset = ref_point['pole_kilo_NEWSS'] - pole_kiro_twins
    print(f"基準にする電柱の検測キロ程: {pole_kiro_twins}km")
    print(f"　　　キロ程のオフセット値: {pole_kiro_offset}km")
    return pole_kiro_offset


def get_Kiro_NEWSS(df_tdm, kiro_offset_dict):
    df_tdm['KiroTei_NEWSS'] = df_tdm.apply(lambda row: row['KiroTei'] + kiro_offset_dict[row['SokuteiDate']], axis=1)
    return df_tdm


def set_imgKiro(config, dir_area, df_tdm):
    imgKilo = {}
    for camera_num in config.camera_types:
        # カメラフォルダ内の画像ファイルを取得
        image_dir = f"{config.image_dir}/{dir_area}/{camera_num}/"
        print(image_dir)
        list_images = helpers.list_images(image_dir)
        print(f"Image counts:{len(list_images)}")
        # 画像ファイル名に対応するキロ程を抽出して辞書型で記録する
        imgKilo_temp = {}
        # imgKilo_temp_values = {}
        for fname in tqdm(list_images):
        # for fname in list_images:
            image_name = re.split('[./]', fname)[-2]
            df_tdm_Series = df_tdm[df_tdm[f"GazoFileName{camera_num}"] == image_name].copy()
            if df_tdm_Series.empty:
                continue
            else:
                DenchuNo = df_tdm_Series['DenchuNo'].item()
                KiroTei = df_tdm_Series['KiroTei_NEWSS'].round(4).item()

            # imgKilo_temp_values["DenchuNo"] = DenchuNo
            # imgKilo_temp_values["KiroTei"] = KiroTei
            # imgKilo_temp[image_name] = imgKilo_temp_values.copy()
            imgKilo_temp[image_name] = {
                "DenchuNo": DenchuNo,
                "KiroTei": KiroTei
            }

        if imgKilo_temp != {}:                                                                       # 2024.6.28 Yamazwa add
            imgKilo[camera_num] = imgKilo_temp.copy()

    return imgKilo

print("準備完了")

2024-08-21 05:28:49.706 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-08-21 05:28:49.707 No runtime found, using MemoryCacheStorageManager
2024-08-21 05:28:49.709 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems

準備完了


## 2. 車モニデータを利用して画像ごとのキロ程マスターデータ(JSON)を作成する
```
ファイルパスなどを設定する
車モニのマスターデータは検測キロ程になっているため、NEWSSキロ程に合わせる基準を設定する
```

In [4]:
# S3にあるCSVファイルを読み込む
bucket_name = 'trolley-monitor'
prefix = 'img-location-table/2024Q1/2024_0001-0180/'
csv_files = list_csv_files(bucket_name, prefix)
print("参考")
print(f"S3にあるTDM(車モニCSV):")
for i, csv_file in enumerate(csv_files):
    print(f"{i}> {csv_file}")

参考
S3にあるTDM(車モニCSV):
0> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0001.csv
1> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0002.csv
2> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0003.csv
3> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0004.csv
4> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0005.csv
5> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0006.csv
6> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0007.csv
7> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0008.csv
8> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0009.csv
9> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0010.csv
10> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0011.csv
11> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_2024_0012.csv
12> img-location-table/2024Q1/2024_0001-0180/TDM_GazoFileIndex_20

In [5]:
# まとめて処理用

# 初期設定
images_path = helpers.list_imagespath_nonCache(config.image_dir)
# ref_point = {
#     "EkiCd_NEWSS": 14010,    # 高崎線での初期値 14010: 宮原
#     "pole_num_NEWSS": 1,    # 高崎線での初期値 1: 構本1号柱
#     "pole_kilo_NEWSS": 3.561    # 高崎線での初期値 3.561: NEWSSキロ程3k561m
# }
# print("キロ程変換の基準")
# print(f"> 駅コード: {ref_point['EkiCd_NEWSS']}")
# print(f"> 基準にする電柱：{ref_point['pole_num_NEWSS']}号柱 {ref_point['pole_kilo_NEWSS']}km")

# images_path = helpers.list_imagespath_nonCache(config.image_dir)
# print(f"線区フォルダ: {images_path}")

# # フォルダを選択
# idx = 4
# dir_area = images_path[idx]
# print(f"線区: {dir_area}")

# # カメラ番号は指定
# camera_num = 'HD21'
# print(f"カメラ番号: {camera_num}")

# target_dir = config.image_dir + "/" + dir_area + "/" + camera_num

# # imagesフォルダ内の画像一覧取得
# base_images = helpers.list_images(target_dir)
# image_name = base_images[0].split('/')[-1]
# print(f"画像名: {image_name}")

# meas_year = image_name.split("_")[0]
# meas_idx = image_name.split("_")[1]
# print(f"走行年度: {meas_year}, 年度内通番: {meas_idx}")

# # 車モニのマスターデータのファイルパス
# tdm_fpath = f'{config.tdm_dir}/temp/TDM_GazoFileIndex_{meas_year}_{meas_idx}.csv'
# print(f'{tdm_fpath=}')

# 手持ちの線区フォルダ
images_path = helpers.list_imagespath_nonCache(config.image_dir)

# カラム名をS3から読み込む
columns_csv_key = 'img-location-table/TDM_GazoFileIndex_ColumnsName.csv'
columns = get_columns_from_csv(bucket_name, columns_csv_key)

# meas_list = []
for images_folder_num in images_path:    
    # カメラ番号のリスト
    cams_path = helpers.list_imagespath_nonCache(f"{config.image_dir}/{images_folder_num}")
    
    # 線区フォルダ内の最初の画像ファイル
    target_dir = f"{config.image_dir}/{images_folder_num}/{cams_path[0]}"
    base_images = helpers.list_images(target_dir)
    image_name = base_images[0].split('/')[-1]
    # print(image_name)
    
    meas_year = image_name.split("_")[0]              # 走行年度
    meas_idx = image_name.split("_")[1]               # 年度通番
    meas_senku = images_folder_num.split('_')[0]      # 線区
    meas_kukan = images_folder_num.split('_')[2]      # 区間
    # if images_folder_num.split('_')[3] == "down":     # 線別
    #     meas_senbetsu = 21
    # elif images_folder_num.split('_')[3] == "up":
    #     meas_senbetsu = 22
    # else:
    #     meas_senbetsu = None                              # 他は？？
    # meas_list.append([meas_year, meas_idx, meas_senku, meas_kukan, meas_senbetsu])


    # 手持ち線区フォルダと一致するCSVをS3から探す
    # stop_loop = 0
    for csv_file in tqdm(csv_files):
        if csv_file[-13:-4] == f"{meas_year}_{meas_idx}":
            
            # データフレームを読み込む
            df_tdm = get_df2(bucket_name, csv_file, columns)            # 2024.08.12 とりあえず版 get_df2()
            
            # 走行日・線区ごとのキロ程補正情報の辞書を作成する
            df_tdm, KiroTei_dict = get_KiroTei_dict(config, df_tdm)
            
            # 測定日を取得　※１つしかないはず・・・
            date = df_tdm['SokuteiDate'].unique().item()
            
            # 線別コードを取得
            if images_folder_num.split('_')[3] == "down":
                SenbetsuCd = 21
            elif images_folder_num.split('_')[3] == "up":
                SenbetsuCd = 22
            else:
                SenbetsuCd = None                              # 他は？？
            
            # 基準となる電柱の情報
            ref_point = {
                "EkiCd_NEWSS": eki_code[meas_senku][meas_kukan]["EkiCd_NEWSS"],
                "pole_num_NEWSS": eki_code[meas_senku][meas_kukan]["pole_num_NEWSS"],
                "pole_kilo_NEWSS": eki_code[meas_senku][meas_kukan]["pole_kilo_NEWSS"]
            }
            
            pole_kiro_offset = get_Kiro_offset(df_tdm, ref_point, date)
            
            kiro_offset_dict = {}
            kiro_offset_dict[date] = pole_kiro_offset
            
            # NEWSSキロ程を計算して 列`KiroTei_NEWSS`に記録する
            df_tdm = get_Kiro_NEWSS(df_tdm, kiro_offset_dict)
            
            # 確認用にエクセルに出力する
            df_tdm.to_excel(f"./TDM/temp/df_tdm_{images_folder_num}.xlsx")
            
            # 初期設定の情報を出力
            print(f"画像フォルダ名：{images_folder_num}")
            print(f"基準にする電柱：{ref_point['pole_num_NEWSS']}")
            print(f"　　　　キロ程：{ref_point['pole_kilo_NEWSS']}")
            print(f"検測キロ程ズレ：{round(kiro_offset_dict[date], 3)}")

            imgKilo = set_imgKiro(config, images_folder_num, df_tdm)
            
            if imgKilo != {}:
                # 結果をJSONファイルに記録する
                dir = f"{config.tdm_dir}/{images_folder_num}.json"
                with open(dir, mode="wt", encoding="utf-8") as f:
                    json.dump(imgKilo, f, ensure_ascii=False, indent=2, default=helpers.default)

            print(f"画像ファイルごとのキロ程情報を{dir}に記録しました")
            
            print(df_tdm.info())
            # stop_loop = 1
            break
            
#     if stop_loop == 1:
#         break
            
            
"""
2024.8.12　めも
・必要なCSVファイルのみS3から持ってくる形式に変更（S3にある場合のみ）
・必要なCSVファイルは走行年度と年度内通番から検索
・それを手持ちの線区フォルダで順繰りに実施
"""

  0%|          | 0/180 [00:00<?, ?it/s]

車モニのマスターデータを読み込みます ※少し時間がかかります
complete: download_path='TDM/temp/TDM_GazoFileIndex_2024_0004.csv'


/tmp/ipykernel_1550/3673016867.py:79: DtypeWarning: Columns (23,26,29,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tdm = pd.read_csv(download_path, names=columns, delimiter='|')


車モニのマスターデータを読み込みました
データフレームのサイズ:(83501, 90)
必要な情報だけフィルタリングしました
データフレームのサイズ:(83501, 11)
基準にする電柱の検測キロ程: 37.2817km
　　　キロ程のオフセット値: -0.14269999999999783km
画像フォルダ名：Joban_001_Toride-st_down_20240402_day
基準にする電柱：1
　　　　キロ程：37.139
検測キロ程ズレ：-0.143
imgs/Joban_001_Toride-st_down_20240402_day/HD11/
Image counts:1036


  0%|          | 0/1036 [00:00<?, ?it/s]

...

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

'\n2024.8.12\u3000めも\n・必要なCSVファイルのみS3から持ってくる形式に変更（S3にある場合のみ）\n・必要なCSVファイルは走行年度と年度内通番から検索\n・それを手持ちの線区フォルダで順繰りに実施\n'

## 作成したJSONファイルの確認

In [9]:
import json
jsonPath = config.tdm_dir
# jsonFile = "Joban_001_Toride-st_down_20240402_day.json"
jsonFile = "Joban_002_Toride-Fujishiro_down_20240402_day.json"
FullPath = jsonPath + "/" + jsonFile
print(FullPath)

jsonOpen = open(FullPath, 'r')
tdmFile = json.load(jsonOpen)

# print(jsonLoad)

TDM/Joban_002_Toride-Fujishiro_down_20240402_day.json


In [11]:
jsonFile.split('.')[0]

'Joban_002_Toride-Fujishiro_down_20240402_day'

In [12]:
import pandas as pd
from openpyxl import Workbook
# import openpyxl

# writer = pd.ExcelWriter('Takasaki_99_Omiya-Kitaageo_down_20230509_day.xlsx', engine='openpyxl')
# wb = openpyxl.Workbook()

jsonname = jsonFile.split('.')[0]

with pd.ExcelWriter(f'{jsonname}.xlsx') as writer:

    for camera in ['HD11', 'HD12', 'HD21', 'HD22', 'HD31', 'HD32']:

        Index, Kirotei, KiroDiff, PoleNum, NewPole = [], [], [], [], []
        for ix, file in enumerate(tdmFile[camera]):
            if ix == 0:
                last_pole = tdmFile[camera][file]['DenchuNo']
                last_kiro = tdmFile[camera][file]['KiroTei']

            Index.append(ix)

            Kirotei.append(tdmFile[camera][file]['KiroTei'])

            KiroDiff.append(tdmFile[camera][file]['KiroTei'] - last_kiro)

            PoleNum.append(tdmFile[camera][file]['DenchuNo'])

            if tdmFile[camera][file]['DenchuNo'] != last_pole:
                NewPole.append('True')
            else:
                NewPole.append('False')

            last_pole = tdmFile[camera][file]['DenchuNo']
            last_kiro = tdmFile[camera][file]['KiroTei']

        df_json = pd.DataFrame({'Index': Index, 'PoleNum': PoleNum, 'NewPole': NewPole, 'Kirotei': Kirotei, 'KiroDiff':KiroDiff})
        df_json.to_excel(writer, sheet_name=camera, index=False)
        # df_json.to_excel(wb, sheet_name=camera, index=False)

# Excelファイルを保存
# writer.save()
# wb.save('Takasaki_99_Omiya-Kitaageo_down_20230509_day.xlsx')


In [36]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

source = ColumnDataSource(df_json)

TOOLTIPS = [
    ('Index', '@ix'),
    ('PoleNum', '@PoleNum'),
    ('Kirotei', '@Kirotei')
]

p = figure(
    plot_width=2000, 
    plot_height=600,
    tooltips=TOOLTIPS
)
     
p.circle(
    source=source, 
    # x='ix',
    # y='kiro'
    x='Index',
    y='Kirotei'
)

show(p)

Loading BokehJS ...

## 駅・駅間コードのjson作成

In [2]:
# エクセル読み込み
df_ekicode = pd.read_excel("TDM/Senku-Code.xlsx")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
# json元データ作成
eki_code = {}
for Senmei in df_ekicode["Senmei"].unique():
    eki_code[Senmei] = {}
    for idx, Kukan in enumerate(df_ekicode["Kukan"]):
        eki_code[Senmei][Kukan] = {
            "EkiCd_NEWSS":df_ekicode["EkiCd"][idx],
            "pole_num_NEWSS":df_ekicode["DenchuNo"][idx],
            "pole_kilo_NEWSS":df_ekicode["KiroTei"][idx]
        }

In [10]:
# jsonファイルに書き出し
path = f"{config.tdm_dir}/eki_code.json"
with open(path, mode="wt", encoding="utf-8") as f:
    json.dump(eki_code, f, ensure_ascii=False, indent=2, default=helpers.default)

In [116]:
# jsonファイル確認
path = path = f"{config.tdm_dir}/eki_code.json"
with open(path, encoding="utf-8") as f:
    eki_code_rd =  json.load(f)

In [117]:
eki_code_rd["Takasaki"].keys()

dict_keys(['Omiya-Miyahara', 'Miyahara-st', 'Miyahara-Ageo', 'Ageo-st', 'Ageo-Kitaageo', 'Kitaageo-st', 'Kitaageo-Okegawa', 'Okegawa-st', 'Okegawa-Kitamoto', 'Kitamoto-st', 'Kitamoto-Kounosu', 'Kounosu-st', 'Kounosu-Kitakounosu', 'Kitakounosu-st', 'Kitakounosu-Fukiage', 'Fukiage-st', 'Fukiage-Gyouda', 'Gyouda-st', 'Gyouda-Kumagaya', 'Kumagaya-st', 'Kumagaya-KumagayaFT', 'KumagayaFT', 'KumagayaFT-Kagohara', 'Kagohara-st', 'Kagohara-Fukaya', 'Fukaya-st', 'Fukaya-Okabe', 'Okabe-st', 'Okabe-Honjo', 'Honjo-st', 'Honjo-Jinbohara', 'Jinbohara-st', 'Jinbohara-Shinmachi', 'Shinmachi-st', 'Shinmachi-Kitafujioka', 'Kitafujioka-st', 'Kitafujioka-Kuragano', 'Kuragano-st', 'Kuragano-TakasakiYD', 'TakasakiYD', 'TakasakiYD-Takasaki', 'Takasaki-st'])